# Learning PageRank via vertical convolution followed with fully connected layers neural networks


In [1]:
import time
from mygraph import MyGraph
from helpers import *

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

## Loading the dataset from numpy files

In [3]:
# Pageank as the target or label data
target_pagerank = np.load("fb_co_pages_pageranks.npy", mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
target_pagerank[0]

array([9.98507331e-05])

In [4]:
# scaling the PageRank data
target_pagerank = target_pagerank * 10000
target_pagerank[0]

array([0.99850733])

In [5]:
target_pagerank.shape

(14113, 1)

In [6]:
# the NFDC matrix as the feature set
matrix_1 = np.load("fb_co_pages_NDFC_matrix_r1-30_sta1_max50_rad5.npy", mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
matrix_1.shape

(14113, 6, 17)

In [7]:
matrix_1 = matrix_1.reshape(14113, 1, 6, 17)
matrix_1.shape

(14113, 1, 6, 17)

# Converting the data into pytorch tenors

In [8]:
features = torch.from_numpy(matrix_1)
targets = torch.from_numpy(target_pagerank)
features.shape, targets.shape

(torch.Size([14113, 1, 6, 17]), torch.Size([14113, 1]))

In [9]:
# Ativating the automatic gradient 
features.requires_grad_(True)
targets.requires_grad_(True)

tensor([[0.9985],
        [0.8875],
        [0.5463],
        ...,
        [0.2811],
        [0.4245],
        [0.3886]], dtype=torch.float64, requires_grad=True)

In [10]:
# Shuffling and dividing the indecies
n_samples = features.shape[0]
n_test = 4113
shuffled_ind = torch.randperm(n_samples)
train_ind = shuffled_ind[:-n_test]
test_ind = shuffled_ind[-n_test:]
# Dividing features and targets into tain and test sets
train_features = features[train_ind]
test_features = features[test_ind]
train_targets = targets[train_ind]
test_targets = targets[test_ind]
train_features.shape, test_features.shape, train_targets.shape, test_targets.shape

(torch.Size([10000, 1, 6, 17]),
 torch.Size([4113, 1, 6, 17]),
 torch.Size([10000, 1]),
 torch.Size([4113, 1]))

## A function for shuffling and dividing train data into batches in each epoch

In [11]:
# shuffling and dividing train_features and train_targets into batches
def next_batch(train_features, train_targets, batch_size=100):
    num_features = train_features.shape[0]
    # Shuffling
    shuffled_ind = torch.randperm(num_features)
    shuffled_train_features = train_features[shuffled_ind]
    shuffled_train_targets = train_targets[shuffled_ind]
    # dividing
    i = 0
    while i < num_features:
        i += batch_size
        yield (shuffled_train_features[i-batch_size:i], shuffled_train_targets[i-batch_size:i])
    return

## The feedforward neural network model

In [12]:
# The Feedforward Neural Network 
class FFNN_model(nn.Module):
    def __init__(self):
        super().__init__()
        # Convolutional layers
        final_out_channels = 6
        self.conv_layer_1 = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=(3,1), stride=(1, 1), padding=(1,0),  padding_mode='zeros')
        self.conv_layer_2 = nn.Conv2d(in_channels=3, out_channels=final_out_channels, kernel_size=(4,1), stride=(1, 1), padding=(1,0),  padding_mode='zeros')
        self.pooling_1 = nn.MaxPool2d(kernel_size=(3,1), stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)
        # Linear layers
        num_features = features.shape[-1] * final_out_channels
        self.fc1 = nn.Linear(num_features, 400)
        self.fc2 = nn.Linear(400, 800)
        self.fc3 = nn.Linear(800, 200)
        self.fc4 = nn.Linear(200, 64)
        self.fc5 = nn.Linear(64, 8)
        self.fc6 = nn.Linear(8, 1)
        self.dropout1 = nn.Dropout(0.4)
        self.dropout2 = nn.Dropout(0.3)
        self.dropout3 = nn.Dropout(0.5)
    
    def forward(self, X):
        X = torch.tanh(self.conv_layer_1(X))
        X = self.pooling_1(X)
        X = torch.relu(self.conv_layer_2(X))
        X = torch.flatten(X, start_dim=1, end_dim=-1)
        X = torch.tanh(self.fc1(X))
        X = torch.relu(self.fc2(X))
        X = self.dropout1(X)
        X = torch.relu(self.fc3(X))
        X = self.dropout3(X)
        X = torch.relu(self.fc4(X))
        X = self.dropout2(X)
        X = torch.tanh(self.fc5(X))
        return self.fc6(X)

In [13]:
# Instantiation of the model
torch.manual_seed(42)
model = FFNN_model()
model

FFNN_model(
  (conv_layer_1): Conv2d(1, 3, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0))
  (conv_layer_2): Conv2d(3, 6, kernel_size=(4, 1), stride=(1, 1), padding=(1, 0))
  (pooling_1): MaxPool2d(kernel_size=(3, 1), stride=(3, 1), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=102, out_features=400, bias=True)
  (fc2): Linear(in_features=400, out_features=800, bias=True)
  (fc3): Linear(in_features=800, out_features=200, bias=True)
  (fc4): Linear(in_features=200, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=8, bias=True)
  (fc6): Linear(in_features=8, out_features=1, bias=True)
  (dropout1): Dropout(p=0.4, inplace=False)
  (dropout2): Dropout(p=0.3, inplace=False)
  (dropout3): Dropout(p=0.5, inplace=False)
)

In [14]:
num_para = 0
for param in model.parameters():
    print(param.numel())
    num_para += param.numel()
print("----------------------")
print(f'Number of all parameters: \n{num_para}')

9
3
72
6
40800
400
320000
800
160000
200
12800
64
512
8
8
1
----------------------
Number of all parameters: 
535683


In [15]:
# Objective function and optimmizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

## Defining the Training Loop

In [16]:
def training_loop(n_epochs=1000,
                  batch_size=100,
                  optimizer=optimizer, 
                  model=model, 
                  loss_fn=criterion, 
                  train_features=train_features, 
                  test_features=test_features, 
                  train_targets=train_targets, 
                  test_targets=test_targets):
    num_features = train_features.shape[0]
    start_time = time.time()
    all_train_loss, all_test_loss = np.zeros(n_epochs), np.zeros(n_epochs)
    for epoch in range(1, n_epochs +1):
        # Training: 
        epoch_losses = []
        # looping through batches
        for train_features, train_targets in next_batch(train_features=train_features, 
                                                        train_targets=train_targets, batch_size=batch_size): 
            train_preds = model(train_features.float())
            train_loss = loss_fn(train_targets.float(), train_preds.float())
            optimizer.zero_grad()
            train_loss.backward(retain_graph=True)
            optimizer.step()
            
            epoch_losses.append(train_loss.item())
        average_epoch_loss = sum(epoch_losses)/len(epoch_losses)
        
        # Test:
        with torch.no_grad():
            test_preds = model(test_features.float())
            test_loss = loss_fn(test_targets.float(), test_preds.float())
        
        all_train_loss[epoch - 1] = average_epoch_loss
        all_test_loss[epoch - 1] = test_loss.item()
        # Printing the result: 
        if epoch == 1 or epoch % 100 == 0:
            print(f"EPOCH: {epoch:{7}}")
            print(f"MEAN TRAIN LOSS:   {average_epoch_loss:.11f},    Test LOSS:   {test_loss.item():.11f}")
            print("-----------------------------------------")
    print("The total time = ", np.round(time.time() - start_time, 3), " seconds!")
    return all_train_loss, all_test_loss

# Training the model

In [17]:
losses = training_loop(n_epochs=2000,
                  batch_size=400,
                  optimizer=optimizer, 
                  model=model, 
                  loss_fn=criterion, 
                  train_features=train_features, 
                  test_features=test_features, 
                  train_targets=train_targets, 
                  test_targets=test_targets)

EPOCH:       1
MEAN TRAIN LOSS:   0.59045001388,    Test LOSS:   0.70292574167
-----------------------------------------
EPOCH:     100
MEAN TRAIN LOSS:   0.14982925355,    Test LOSS:   0.49151390791
-----------------------------------------
EPOCH:     200
MEAN TRAIN LOSS:   0.06843725592,    Test LOSS:   0.42770397663
-----------------------------------------
EPOCH:     300
MEAN TRAIN LOSS:   0.03769408911,    Test LOSS:   0.39609655738
-----------------------------------------
EPOCH:     400
MEAN TRAIN LOSS:   0.02580842562,    Test LOSS:   0.37151297927
-----------------------------------------
EPOCH:     500
MEAN TRAIN LOSS:   0.01825783402,    Test LOSS:   0.35698744655
-----------------------------------------
EPOCH:     600
MEAN TRAIN LOSS:   0.01740817353,    Test LOSS:   0.34787666798
-----------------------------------------
EPOCH:     700
MEAN TRAIN LOSS:   0.01075147185,    Test LOSS:   0.34239444137
-----------------------------------------
EPOCH:     800
MEAN TRAIN LOSS: 

In [18]:
# Saving the page rank as a numpy array into a file
np.save("pagerank_train_losses_001_conv.npy", losses[0], allow_pickle=False, fix_imports=True)

In [19]:
# Saving the page rank as a numpy array into a file
np.save("pagerank_test_losses_001_conv.npy", losses[1], allow_pickle=False, fix_imports=True)

In [20]:
model.eval()
errors = []
for i in range(n_test):
    targ = test_targets[i].item()
    feat = test_features[i].float().view(1,1,6,17)
    pred = model(feat).item()
    inaccuracy = abs(1 - pred/targ) * 100
    errors.append(inaccuracy)
    if i%100 == 0:
        print(f"target {targ},    prediction: {pred}\nindex {i*40}:       inaccuracy: {np.round(inaccuracy, 3)}%")
        print("-----------------------------------")

target 0.2619875960369246,    prediction: 0.41712671518325806
index 0:       inaccuracy: 59.216%
-----------------------------------
target 1.096927690930318,    prediction: 1.0511575937271118
index 4000:       inaccuracy: 4.173%
-----------------------------------
target 1.5850305319179239,    prediction: 1.0189944505691528
index 8000:       inaccuracy: 35.711%
-----------------------------------
target 0.6794130477008715,    prediction: 0.386701762676239
index 12000:       inaccuracy: 43.083%
-----------------------------------
target 0.18066128641573986,    prediction: 0.18301735818386078
index 16000:       inaccuracy: 1.304%
-----------------------------------
target 0.8911234807504349,    prediction: 0.9016146659851074
index 20000:       inaccuracy: 1.177%
-----------------------------------
target 0.6387625647351677,    prediction: 0.2668488323688507
index 24000:       inaccuracy: 58.224%
-----------------------------------
target 0.22504328229655007,    prediction: 0.48876696825

In [21]:
print("Average inaccuracy: ", np.round(sum(errors)/len(errors), 3))

Average inaccuracy:  35.965


In [22]:
torch.save(model, "trained_conv_model_pagerank_1.pt")